#Policy Based Methods

Mario Fiorino

$\;$

**Introduzione**

I metodi presentati nei notebook precedenti ("action-value methods") si basano sulla funzione $Q(s,a)$ valore-azione. Ovvero:

- Si apprendere la funzione $Q(s,a)$ attraverso l'interazione con l'ambiente.  

- Una volta appresa tale la funzione, l'agente seleziona l'azione da intraprendere nello stato corrente $s$ basandosi sui valori stimati di $Q(s,a)$. Tipicamente, viene scelta l'azione che massimizza il valore stimato.

-  Tali metodi non esplicitano una funzione policy$: a → s $, questa è implicitamente definita dai valori di $Q(s,a)$.

$\;$

I metodi policy based invece:

- Apprendono una funzione di policy, parametrizzata; ad esempio una ANN.

- Così strutturate, tale funzione di policy in grado di selezionare azioni senza consultare una funzione $Q(s,a)$



$\;$

Vantaggi

- La policy parametrizzata, in genere, è una funzione più semplice da approssimare, rispetto alla parametrizzazione della funzione valore-azione.

- Stochastic policies : questi metodi semplificano le cosa quando si tratta di apprendere policy che selezionano azioni in modo probabilistico (utili in scenari con incertezza). Ad esempio nel gioco Sasso-carta-forbici(Rock-Paper-Scissors), la policy ottimale da ricavare è di tipo stocastico.

- Efficacia in "high-dimensional" spazi ed/o in spazi ad azione continua.

- Infine, la parametrizzazione della policy è un buon modo per sfruttare la conoscenza pregressa nella definizione della forma desiderata della policy.

Svantaggi:


- Tali metodi tendono a convergere verso un locale ottimo, piuttosto che verso l'ottimo globale.

- La valutazione della performance di una policy può essere inefficente e presentare un'alta varianza.

## Idee di base




Notazioni:


$\textbf{θ}$ è il vettore dei parametri della policy $\pi_\theta$ , con dimensioni $d'$. Ovvero : $\textbf{θ} \in R^{d'}$

$\pi(a|s, \textbf{θ}) = Pr \{ A_t =a | S_t =s, \textbf{θ}_t = \textbf{θ} \} \; \; $  indica la probabilità che l'azione $a$ venga intrapresa nello stato $s$ al tempo $t$, con i parametri della policy uguali $\textbf{θ}$.

$J(\textbf{θ})$ è una funzione, spesso chiamata **objective function** (funzione obiettivo) , che misura le performance policy ${\pi_\theta}$, dipendente dalla variazione dei suoi parametri. Ad esempio negli ambienti episodici, un modo molto semplice per valutare tale rendimento, è quello di scegliere la value fuction della policy nello stato di partenza, cioè expected return ottenuto seguendo tale policy,  partendo da uno specifico stato iniziale :
$J(\textbf{θ}) = V_{\pi_\theta} (s_{0}) = \mathbb{E}_{ {\pi_\theta}} [ \sum_{t=0}^{T} r_t | s_0] $ .
Si noti, in questo esempio si assume che il discount factor $\gamma = 1 $ e si considera un numero finito di episodi. $r_t$ è il reward ottenuto ad ogni step seguendo una certa traiettoria di eventi stato-azione campionata da $\pi_\theta$. $T$ è lo step nel Terminal state.

Sia  $\tau$ un traiettoria di eventi stato-azione campionata da $\pi_\theta$ che parte da un stato $s_0$, allora

$  \mathbb{E}_{ {\pi_\theta}} [ \sum_{t=0}^{T} r_t | s_0] = \mathbb{E}_{\tau\sim \pi_θ} [R(\tau)] = \int_{\tau} P(\tau|\pi_\theta) R(\tau) $

Dove $R(\tau)$ è la somma dei premi ottenuti da $\tau$.
Mentre $P(\tau|\pi_\theta)$ è la probabilità di estrarre $\tau$ dalla policy $\pi_\theta$

$\;$

**Obiettivo**:

Apprendere i parametri $\textbf{θ}$ che massimizzano $J(\textbf{θ})$.

$\;$

Come?

Per risolvere tale problema, si possono impiegare diverse strategie, le più comuni si suddividono in due categorie:

Metodi non basati sul gradiente:

- Hill climbing (Questo metodo esplora lo spazio dei parametri in modo iterativo, muovendosi nella direzione che aumenta i valori della funzione $J$)

- Simplex / amoeba / Nelder Mead

- Genetic algorithms

Metodi basati sul gradiente:

- Gradient descent

- Conjugate gradient

- Quasi-newton


In questo notebook, ci concentriamo specificamente sul gradient descent (discesa del gradiente), data la sua ampia diffusione e la sua efficacia in molti scenari.


## Policy Gradient


$\;$


L'idea di fondo dei metodi policy gradient è quello di ottimizzare la policy del sistema incrementando la probabilità di azioni che conducono a rendimenti più elevati, e diminuendo la probabilità di azioni associate a rendimenti inferiori.

$\;$


Data una certa distribuzione $\rho$ da cui sono campionati gli stati iniziali $s_0$; consideriamo la seguente funzione obiettivo, che sia una funzione differenziabile in $\mathbf{θ}$:

$J(\textbf{θ}) =  \mathbb{E}_{ {s_0 \sim \rho}} [ V_{\pi_\theta} (s_0)] $

Gli algoritmi policy gradient si prefiggono di individuare quei valori ottimali dei parametri $\mathbf{θ^{*}}$ che definiscono la policy $\pi_\mathbf{θ}$, con l'obiettivo di massimizzare la funzione di valore $J(\mathbf{θ})$, spesso raggiungendo un massimo locale.

Per realizzare ciò, questi algoritmi impiegano la tecnica del *stochastic gradient ascent*, aggiornando così i parametri:

$\mathbf{\theta_{k+1}} = \mathbf{\theta_k} + \alpha \nabla_{\mathbf{\theta}} J(\mathbf{\theta_k}) \; \;$ $\; \;$  Nota Formale, limitazioni di LaTeX nel notebook: se pur nell'equazione esposta, i parametri $\mathbf{{\theta_k}}$ non sono rappresentati in grassetto, sono sempre da pensarsi come vettori di dimensione $d'$.

$\;$

**Problema** : Come ricavare  $\nabla_{\theta} J(\textbf{θ})$ ?

Il calcolo del gradiente è complicato perché, sia la selezione dell'azione ( questa è la parte fattibile ) sia la distribuzione  degli stati in cui vengono effettuate tali selezioni (qui ci sono delle complicazioni in quanto tale distribuzione dipende sia dalla policy che dal modello dell’ambiente, che è tipicamente sconosciuto in un contesto RL model-free) dipendono dalle variazioni dei parametri $\textbf{θ}$ della policy.


In pratica, poiché la probabilità delle varie traiettorie ottenute dalla policy $\pi_{\mathbf{θ}}$ dipende anche dal modello incognito dell'ambiente, non possiamo calcolare direttamente il gradiente $\nabla_{\theta} J(\mathbf{θ})$.

$\;$

###Soluzione: **Policy Gradient Theorem**

Il teorema Policy Gradient fornisce un'espressione analitica computazionalmente fattibile del gradiente in questione:


$\nabla_{\theta} J(\mathbf{θ}) \propto  \mathbb{E}_{ {s \sim d^{\pi_\theta} _{\rho}} ; \;a \sim {\pi_\theta} } [  Q_{\pi_{θ}}(s,a) \nabla_{\theta} \log \pi_{θ}(a|s )] $


Dove:

Il simbolo $\propto$ significa “proporzionale a”.

$d^{\pi_\theta} _{s_0} (s)$ è la *state visitation distribution* che praticamente descrivie la probabilità (o frequenza) di finire in un certo stato $s$, partendo da uno stato iniziale $s_0$, è seguendo la policy $\pi_\theta$. Per maggiori info (Equivalent Formulations of Policies and Reward) : https://www.alexirpan.com/rl-derivations/

$d^{\pi_\theta} _{\rho}  = \mathbb{E}_{s_0 \sim ρ} [ d^{\pi_\theta} _{s_0} (s)]$

$\;$

Per la verifica analitico-matematica del teorema:
Sutton & Barto,2018, PDF 347

oppure: https://users.ece.cmu.edu/~yuejiec/ece18813B_notes/lecture12-policy-optimization.pdf


$\;$

In pratica, il teorema esprime una relazione proporzionale tra il gradiente della policy $\nabla_{\theta} \pi(a|s)$ (un vettore colonna che possiamo campionare) ed il gradiente $ \nabla_{\theta} J(\mathbf{θ})$.

Si noti:

Così espresso, il gradiente di $J$ può essere rappresentato come expectation $\mathbb{E}[ \; ]$. Significa che possiamo usare un campionamento per approssimarlo.



$\;$



Formulazione Generale:

Sia $X$ una variabile casuale, e data una funzione di densità di probabilità  $p_θ(X)$. Si definisce funzione obiettivo $J(θ)$:


$J(θ)=\mathbb{E}[f(X)]=∫_x f(x)p_{θ} (x)dx$


Per $f(x)$ funzione arbitraria che non dipende da $θ$


Vogliamo calcolare $∇_θ J(θ)$. Con delle manipolazioni matematiche, possiamo cambiare il gradiente dell'expectation nell'expectation del gradiente:

$ ∇_θ J(θ)=∫_x f(x) ∇_θ p_θ(x) dx = \mathbb{E}[f(x)∇_θ \log(p_θ(x) )]$

Supponiamo ora che la variabile casuale sia una traiettoria $τ=(s_0,a_0,s_1,a_1,s_2,⋯)$, generata dalla policy $π_θ$ in un MDP. Sia $f(τ)$ il return lungo tale traiettoria. $J(θ)$ è ora l'expected return della policy, ed il suo gradiente offre un modo per aggiornare la policy verso una maggiore ricompensa.

Si osservi infine, che avremmo potuto scegliere anche un'altra funzione obiettivo, diversa da $ \mathbb{E}_{ {s_0 \sim \rho}} [ V_{\pi_\theta} (s_0)] $, il risultato finale del Policy Gradient Theorem è indipendente da questa scelta.


$\;$

Ref:



Compendio Policy Gradient Algorithms, chiaro e completo:

https://lilianweng.github.io/posts/2018-04-08-policy-gradient/


Corso di David Silver:

https://www.youtube.com/watch?v=KHZVXao4qXs


Elliot Waite - Machine learning video; un ottima spiegazione con un esempio pratico:

https://www.youtube.com/watch?v=cQfOQcpYRzE



Sutton & Barto, Reinforcement learning, An Introduction, 2018.

https://jonathan-hui.medium.com/rl-policy-gradients-explained-9b13b688b146

##REINFORCE (aka Monte Carlo Policy Gradient)


REINFORCE = REward Increment = Non-negative Factor × Offset Reinforcement × Characteristic Eligibility


L'algoritmo è stato introdotto da Ronald J. Williams nel 1992.



$\;$

Idee chiave:

- Aggiornare i parametri tramite la tecnica stochastic gradient-ascent, sfruttando il teorema Policy Gradient per ricavare $\nabla_{\theta} J(\mathbf{θ})$

- Viene impiegato il classico Return completo Monte Carlo, $G_t$:  cioè a partire dallo step $t$, tale valore include tutte le ricompense future ottenute dalla policy fino al termine dell'episodio. Nella formulazione ottenuta dal teorema Policy Gradient, questo $G_t$ sostituisce $Q_{\pi}(s,a)$


$\;$

Regola di update dei parametri della policy:

$\mathbf{\theta}_{t+1} = \theta_t + \alpha \cdot [ G_t\frac{\nabla \pi (A_t|S_t, \theta_t)}{\pi(A_t|S_t, \theta_t)} ] \; \;$ Si ricorda che $\theta$ se pur non in grassetto, è da considerarsi un vettore; e che la notazione $π$  è uno snellimento di  $π_θ$.

Si noti, $\nabla_{\theta} J({θ}) = \mathbb{E}_{\pi} [ G_t \frac{\nabla \pi (A_t|S_t, \theta_t)}{\pi(A_t|S_t, \theta_t)} ] $. In parantesi quadra vi è una quantità campionabile ad ogni time-step la cui expectation è uguale al gradiente della funzione obiettivo.

Sfruttando la legge analitica di derivazione del logaritmo , l'espressione frazionaria viene sostituita da

$∇ \log \pi (A_t|S_t, \theta_t) =  \frac{\nabla \pi (A_t|S_t, \theta_t)}{\pi(A_t|S_t, \theta_t)} $

Questa formulazione logaritmica risutla anche computazionalmente più vantaggiosa.


$\;$

Approfondimenti Teorici

Per un approfondimento sui passaggi algebrici e matematici che portano a questa regola, consultare il testo di Sutton & Barto, 2018, PDF 348 - 349. Sempre in queste pagine viene evidenziata la modifica usata nell'update: $[ G_t\frac{\nabla \pi (A_t|S_t, \theta_t)}{\pi(A_t|S_t, \theta_t)} ]$ , rispetto la formulazione teorica di base che prevede una sommatoria sulle azioni:
*This algorithm, which has been called an all-actions method because its update involves all of the actions, is promising and deserving of further study, but our current interest is the classical REINFORCE algorithm
(Willams, 1992) whose update at time t involves just $A_t$, the one action actually taken at time t.*

Sempre il testo di Sutton & Barto, aggiunge il seguente commento finale riguardo tale formulazione:

*This update has an intuitive appeal. Each increment is proportional to the product of a return $G_t$ and a vector, the gradient of the probability of taking the action actually taken divided by the probability of taking that action. The vector is the direction in parameter space that most increases the probability of repeating the action $A_t$ on future visits to state $S_t$. The update increases the parameter vector in this direction proportional to the return, and inversely proportional to the action probability. The former makes sense because it causes the parameter to move most in the directions that favor actions that yield the highest return. The latter makes sense because otherwise actions that are selected frequently are at an advantage (the updates will be more often in their direction) and might win out even if they do not yield the highest return.*

$\;$




**Convergenza**

Per valori di $α$ sufficentemente piccoli,
è garantito un miglioramento delle prestazioni della policy nella massimizzazione della ricompensa.

La convergenza verso un ottimo locale è garantita con le condizioni di approssimazione stocastica (stochastic approximation conditions) standard, viste più volte in questi notebook, per $α$ decrescente.


$\;$


**Svantaggi**


Come per il metodo Monte Carlo classico, REINFORCE può presentare un'elevata varianza e quindi produrre un apprendimento lento.

Di fattto abbiamo bisogno di ulteriori meccanismi per ridurre la varianza. Un esempio di risposta a tale problema :



REINFORCE with Baseline (Sutton & Barto, 2018, PDF 351)



### Env: Cart Pole (Gym)

$\;$

Nota : In CartPole, l'agente riceve una ricompensa con un valore pari a $+1$ per ogni singolo time-step in cui riesce a mantenere l'equilibrio .

https://www.gymlibrary.dev/environments/classic_control/cart_pole/


https://github.com/openai/gym/wiki/Table-of-environments


In [ ]:
import gym

env=gym.make('CartPole-v1')
#help(env.unwrapped)

episodeNumber=2
max_timeSteps=3


for episodeIndex in range(1,episodeNumber):
    initial_state=env.reset()

    #print("Ep:",episodeIndex)
    #appendedObservations=[]

    for timeIndex in range(1,max_timeSteps+1):
        print("timeIndex=",timeIndex)

        random_action=env.action_space.sample()
        print("random_action=",random_action)

        observation, reward, done,  info = env.step(random_action)

        #appendedObservations.append(observation)

        print("[Cart Position,Cart Velocity,Pole Angle,Pole Angular Velocity]:",observation)
        print("reward=",reward)
        print("done=",done)
        print("info=",info)
        print("")

        if done:
            break

env.close()

timeIndex= 1
random_action= 0
[Cart Position,Cart Velocity,Pole Angle,Pole Angular Velocity]: [-0.03882672 -0.17139114  0.04385891  0.34949616]
reward= 1.0
done= False
info= {}

timeIndex= 2
random_action= 1
[Cart Position,Cart Velocity,Pole Angle,Pole Angular Velocity]: [-0.04225454  0.02308048  0.05084883  0.07095963]
reward= 1.0
done= False
info= {}

timeIndex= 3
random_action= 0
[Cart Position,Cart Velocity,Pole Angle,Pole Angular Velocity]: [-0.04179293 -0.1727322   0.05226803  0.37924212]
reward= 1.0
done= False
info= {}



###Moduli ed Inizializzazione ambiente

In [ ]:
import gym
import numpy as np
from collections import deque
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (16, 10)

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
torch.manual_seed(0)

import base64, io

# For visualization
from gym.wrappers.monitoring import video_recorder
from IPython.display import HTML
from IPython import display
import glob

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

env = gym.make('CartPole-v1')

### Premesse tecniche alla Policy come ANN

La **funzione softmax** trasforma un vettore di numeri reali in una distribuzione di probabilità.
In pratica, applica una funzione esponenziale a ciascun elemento, rendendolo positivo, e poi divide per la somma di tutti i valori esponenzializzati, garantendo che la somma delle probabilità risulti pari a uno.

In questo scenario, dove si sono solo due azioni discrete (le nostre "classi"), la funzione softmax produce una distribuzione di probabilità su queste due azioni.

Per spazi di azioni discreti, la funzione softmax rappresenta una scelta comune.

In [ ]:
# Funzione softmax in Pytorch
# dim (int) – A dimension along which Softmax will be computed (so every slice along dim will sum to 1)

softmax0 = torch.nn.Softmax(dim=0) # Applies along columns
softmax1 = torch.nn.Softmax(dim=1) # Applies along rows

v = np.array([[1,2,3],
              [1,5,0]])
v =  torch.from_numpy(v).float()

print(softmax0(v))
print("")
print(softmax1(v))


tensor([[0.5000, 0.0474, 0.9526],
        [0.5000, 0.9526, 0.0474]])

tensor([[0.0900, 0.2447, 0.6652],
        [0.0179, 0.9756, 0.0066]])


In [ ]:
#Categorical in pytorch

#La classe Categorical in PyTorch definisce una distribuzione di probabilità discreta.
#Puoi specificare la distribuzione in due modi (ma non contemporaneamente):
#Probabilità (probs): Tramite un tensore contenente le probabilità relative di ciascuna categoria. La somma degli elementi di questo tensore deve essere pari a 1 (lungo la dimensione specificata durante il campionamento).
#Logit (logits): Tramite un tensore contenente i logit per ciascuna categoria.

# Se probs è unidimensionale con lunghezza K, ciascun elemento rappresenta la probabilità relativa di campionare la categoria corrispondente all'indice.
# ovvero i campioni saranno interi da {0,…,K−1}

#https://pytorch.org/docs/stable/distributions.html

#log_prob
#Restituisce semplicemente il logaritmo della probabilità che si verifichi un determinato campione (o categoria) estratto dalla distribuzione.
#https://en.wikipedia.org/wiki/Log_probability
#https://chrispiech.github.io/probabilityForComputerScientists/en/part1/log_probabilities/

print("E1")
m = Categorical(torch.tensor([ 0.25, 0.25, 0.25, 0.25 ]))
s = m.sample()
print(s)  # ha un uguale probabilità di campionare 0, 1, 2, 3
print(m.log_prob(s))
print(m.log_prob(torch.from_numpy(np.ones(1)*3)))

print("E2")
m = Categorical(torch.tensor([ 0.9999, 0.00001 ]))
s = m.sample()
print(s)  # quasi sempre campiona 0
print(m.log_prob(s))
print(m.log_prob(torch.from_numpy(np.ones(1))))

print("E3")
m = Categorical(torch.tensor([ 0.33, 0.33, 0.33, 0.01 ]))
s = m.sample()
print(s)  # di rado campiona 3
print(m.log_prob(s))
print(m.log_prob(torch.from_numpy(np.ones(1)*3)))

E1
tensor(0)
tensor(-1.3863)
tensor([-1.3863])
E2
tensor(0)
tensor(-1.0014e-05)
tensor([-11.5128])
E3
tensor(1)
tensor(-1.1087)
tensor([-4.6052])


###Policy come ANN



I metodi basati sulle policy, producono in uscita una certa distribuzione di probabilità sulle azioni disponibili. Ad esempio, se si hanno solo due azioni discrete in un certo stato, potrebbe concretizzarzi in uscita una distribuzione del tipo $[0.2 \;\; 0.8]$.


$\;$

Si noti: per garantire l’esplorazione e quindi l'apprendimento, si richiede che la policy non diventi mai puramente deterministica.


In [ ]:
#Codice ispirato :
#https://github.com/goodboychan?tab=repositories

class Policy(nn.Module):

    #Definiamo una ANN a due layer fully connected con relative activation function
    def __init__(self, state_size=4, action_size=2, hidden_size=32):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden_size) # Matrice pesi fc1.weight: torch.Size([32, 4])
        self.activation1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, action_size) # Matrice pesi fc2.weight: torch.Size([2, 32])
        self.activation2 = torch.nn.Softmax(dim=1) # Nota per dim=0 da chiaramente [[1., 1.]]

    def forward(self, state):
        x = self.fc1(state)
        x1 = self.activation1(x)
        x2 = self.fc2(x1) #è una roba del tipo tensor([[-0.0344,  0.0977]], grad_fn=<AddmmBackward0>)
        x3 = self.activation2(x2)
        return x3

    # Definiamo un modo per campionare un azione e il log della sua probabilità
    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device) # questo rigo evita il TypeError:
        # linear(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

        probs = self.forward(state).cpu()
        model = Categorical(probs)
        action = model.sample() # qui campiona l'azione usando i risultati della ANN
        return action.item(), model.log_prob(action)

In [ ]:
#Testing della classe

p = Policy().to(device)
state = env.reset()

for timeIndex in range(1,2):

        state1 = torch.from_numpy(state).float().unsqueeze(0)
        print("p.forward(state)=", p.forward(state1))

        action, log_prob = p.act(state)
        print("action=",action)
        print("log_prob=",log_prob)

        state, reward, done, _ = env.step(action)


        if done:
            break

env.close()

p.forward(state)= tensor([[0.5816, 0.4184]], grad_fn=<SoftmaxBackward0>)
action= 0
log_prob= tensor([-0.5419], grad_fn=<SqueezeBackward1>)


### Premesse alla REINFORCE Core function


In [ ]:
#Premssa Tecnica : torch.cat
#https://pytorch.org/docs/stable/generated/torch.cat.html

x = torch.randn(1, )
print(x)
y = torch.randn(1, )
print(y)

print(torch.cat([x,y],dim=0))
print( (torch.cat([x,y],dim=0))[0].type)
print("sum=", torch.cat([x,y]).sum())
print("")

####

x = torch.randn(1,4 )
print(x)
print(torch.cat([x, x, x],dim=1))



tensor([0.3001])
tensor([1.3226])
tensor([0.3001, 1.3226])
<built-in method type of Tensor object at 0x798f25ca7330>
sum= tensor(1.6227)

tensor([[-1.2141, -2.2789, -2.6911,  0.0192]])
tensor([[-1.2141, -2.2789, -2.6911,  0.0192, -1.2141, -2.2789, -2.6911,  0.0192,
         -1.2141, -2.2789, -2.6911,  0.0192]])


Buono a sapersi:

Anche se non è il caso di Cart Pole, dove tutti i rewads sono uguali (e non verrà qui applicato); generalmente, per questioni di stabilità, si preferisce normalizzare il return dei rewards. Se si analizza le equazioni di backpropagation, si osserva che il return influenza i gradienti. Per tale ragione, si vuole mantenere i valori campionati del return, in un intervallo specifico.

L' adozione di questa normalizzazione non si basa su solide garanzie teoriche, bensì su considerazioni di natura pratica.

In [ ]:
def normalize_rewards(r):
  # r è la lista contenente i rewads accumulati dell'episodio
  r = torch.tensor(r).to(device)
  print(r.mean())
  print(r.std())
  r = (r - r.mean()) / (r.std() + 1e-9)
  return r # ritorno la lista di reward normalizzati

### REINFORCE Core function

In [ ]:
def list_G_for_each_step(input_list, gamma=1):
    output_list = []
    # Itera in reverse order
    for i in range(len(input_list) - 1, -1, -1):
        # Calcola la somma
        current_sum = sum(input_list[i:])
        output_list.append(current_sum)
        # NOTA: semplifichiamo le cose assumendo che gamma = 1.
        # Altrimenti nell'elaborazione avremmo dovuto considerare anche
        # il fattore discount:  *gamma**(k-t-1))
        # Vedi algoritmo di Sutton&Barto PDF 350
    output_list.reverse()
    return output_list


def reinforce(policy, optimizer, n_episodes=1000, max_t=1000, gamma=1.0, print_every=100):

    scores_deque = deque(maxlen=100)
    scores = []

    for e in range(1, n_episodes+1):
        saved_log_probs = []
        rewards = []
        state = env.reset()

        # Genera una traiettoria
        for t in range(max_t):
            action, log_prob = policy.act(state)
            saved_log_probs.append(log_prob) # è roba del tipo : log_prob= tensor([-0.7196])
            state, reward, done, _ = env.step(action)
            rewards.append(reward)
            #la lista reward cresce in dimensione ad ogni step: rewards [1.0], rewards [1.0, 1.0], rewards [1.0, 1.0, 1.0], ...
            if done:
                break


        #Elabora il return G_t
        G = list_G_for_each_step(rewards)
        #R = sum(rewards)  # un alternativa praticabile

        # Utility
        scores_deque.append(sum(rewards)) # Serve per il debug
        scores.append(sum(rewards)) # Serve per il grafico

        # Calcola la loss
        policy_loss = []
        for j in range(len(G)):
            # Tieni presente che stiamo utilizzando Gradient Ascent, non il Descent.
            # E' una convenzione standard, sia in PyTorch che TF, eseguire la minimizzazione anziché la massimizzazione.
            # Quindi, dobbiamo convertire l'obiettivo da massimizzazione in obiettivo da minimizzazione,
            # semplicemente aggiungendo un segno negativo.
            policy_loss.append(-saved_log_probs[j] * G[j])

        # Varainte algoritmo standard
        # A differenza dell'algortimo proposto da Sutton PDF 350
        # dove update dei parametri è fatto "for each step of the episode"
        # in questo caso  preferisco fare un unico update per ogni episodio
        # sfruttando la somma sulle (log_prod*G) campionate.
        # Per la versione standard suggerisco:
        # https://github.com/mitchellvitez/vanilla-policy-gradient/tree/main
        #
        # Nota: torch.cat(policy_loss).mean()
        # usando tale istruzione le cose non funzionano,
        # ottengo valori ballerini Average Score: 75.22,.. 493.66, 177.89 , ... , 91.63
        # e risulta nei test : "Max reward was taken in  0.0 % of times"
        #
        # Nota : torch.cat
        # Si tenga presente che torch.cat trasforma la lista classica di tensori in un tensore "lista" che contiene tensori
        # cioè, da: policy_loss= [tensor([12.8589], grad_fn=<MulBackward0>), tensor([12.5127], grad_fn=<MulBackward0>), tensor([8.6259],..]
        # a:  policy_loss= tensor([12.8589, 12.5127, 8.6259, ...,  12.4928])

        policy_loss = torch.cat(policy_loss).sum()
        #policy_loss è una roba del tipo : tensor(10.4294, grad_fn=<MeanBackward0>)


        # Backpropagation
        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()


        # Debug
        if e % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(e, np.mean(scores_deque)))

        # Se raggiungi un certo punteggio, esempio 195.0, ferma il processo di training
        #if np.mean(scores_deque) >= 195.0:
        #    print('Environment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(e - 100, np.mean(scores_deque)))
        #    break

    print("\n...End of the training.\n")
    return scores

### Training

In [ ]:
#torch.autograd.set_detect_anomaly(True)
#env.spec.reward_threshold = 600  # the standard threshold for rewards is 500 for v1

policy = Policy().to(device)
optimizer = optim.Adam(policy.parameters(), lr=1e-2)
score = reinforce(policy, optimizer, n_episodes=1_000)


Episode 100	Average Score: 53.81
Episode 200	Average Score: 204.24
Episode 300	Average Score: 266.08
Episode 400	Average Score: 153.34
Episode 500	Average Score: 281.38
Episode 600	Average Score: 258.84
Episode 700	Average Score: 255.41
Episode 800	Average Score: 422.23
Episode 900	Average Score: 494.36
Episode 1000	Average Score: 500.00


###Plot

In [ ]:
# plot the scores
fig = plt.figure(figsize=(4,2))
#ax = fig.add_subplot(111)
ax = plt.gca()
ax.set_ylim([1, 600])
plt.plot(np.arange(1, len(score)+1), score)
plt.ylabel('Score')
plt.xlabel('Episode')
plt.show()

### Test con Video

In [ ]:
def show_video(env_name):
    mp4list = glob.glob('./*.mp4')
    if len(mp4list) > 0:
        mp4 = './{}.mp4'.format(env_name)
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display.display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

def show_video_of_model(policy, env_name):
    env = gym.make(env_name)
    vid = video_recorder.VideoRecorder(env, path="./{}.mp4".format(env_name))
    state = env.reset()
    done = False
    for t in range(1000):
        vid.capture_frame()
        action, _ = policy.act(state)
        next_state, reward, done, _ = env.step(action)
        state = next_state
        if done:
            break
    vid.close()
    env.close()

In [ ]:
show_video_of_model(policy, 'CartPole-v1')

In [ ]:
show_video('CartPole-v1')

### Test su 100 campioni

In [ ]:
episodeNumber=100
max_timeSteps=1_000

cont = 0
max_reward = 500


for episodeIndex in range(1,episodeNumber):
    observation=env.reset()
    rewards = 0

    for timeIndex in range(1,max_timeSteps+1):
        action, _ = policy.act(observation)
        observation, reward, done,  info = env.step(action)
        rewards += reward

        if done:
            if max_reward == rewards:
                  cont += 1

            break

env.close()

print("Max reward was taken in ", round((cont/episodeNumber)*episodeNumber,2), "% of times" )

Max reward was taken in  99.0 % of times
